In [34]:
import pandas as pd
import os
from tqdm import tqdm
import pickle

# 设定文件夹路径
folder_path = os.getcwd()

# 列出所有 CSV 文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

df_list

[       ReqID   Oid   Did  ReqTime  Size
 0          1  2447  2561        0     1
 1          2  2055  1366        0     1
 2          3   892   753        0     1
 3          4  2611  1265        0     1
 4          5  1208  1184        0     1
 ...      ...   ...   ...      ...   ...
 23619  23620   481   726     7198     1
 23620  23621   203  1398     7198     1
 23621  23622  2521  2018     7199     1
 23622  23623  2375  2686     7199     1
 23623  23624   129  2135     7199     6
 
 [23624 rows x 5 columns],
        ReqID   Oid   Did  ReqTime  Size
 0          1   632  2350        0     2
 1          2   871     2        0     1
 2          3    19    68        0     1
 3          4  1248  2558        0     1
 4          5   746   716        1     1
 ...      ...   ...   ...      ...   ...
 21984  21985  2026  1616     7197     2
 21985  21986  1985  2693     7198     2
 21986  21987  2085  1116     7198     2
 21987  21988  1356  1194     7198     1
 21988  21989  3110  1438   

In [35]:
len(df_list)

10

In [36]:
PATH_MANHATTAN_ALL_PATH_TIME_MATRIX = "../NYC_Manhattan_AllPathTimeMatrix.pickle"
with open(PATH_MANHATTAN_ALL_PATH_TIME_MATRIX, 'rb') as t:
    ALL_PATH_TIME_MATRIX = pickle.load(t)
# 用于存储更新后的DataFrame
adjusted_dfs = []
# 初始化一个变量来跟踪最后的ReqTime
last_req_time = 0

for i, df in enumerate(df_list):
    if i == 0:
        # 第一个DataFrame不需要调整
        adjusted_dfs.append(df)
        last_req_time = df['ReqTime'].max()
    else:
        # 更新ReqTime
        df['ReqTime'] = df['ReqTime'] + last_req_time + 1
        adjusted_dfs.append(df)
        last_req_time = df['ReqTime'].max()

# 合并所有DataFrame
result_df = pd.concat(adjusted_dfs, ignore_index=True)


# Reset ReqID
result_df.drop(result_df.columns[0], axis=1, inplace=True)
result_df.insert(0, "ReqID", range(1, len(req) + 1))

# Delete requests that can be completed within 3 minutes(too short)
for index, row in tqdm(result_df.iterrows()):
    reqID = row['ReqID']
    Oid = row['Oid']
    Did = row['Did']
    timeCost = ALL_PATH_TIME_MATRIX[Oid][Did]
    if timeCost <= 180:
        result_df.drop(index, inplace=True)

result_df.to_csv('combined_file.csv', index=False)

226676it [02:12, 1717.03it/s]


In [2]:
import pandas as pd
# prune reqs with size > 3:
req_matrix = pd.read_csv('combined_file.csv')
for index, row in req_matrix.iterrows():
    if row["Size"] > 3:
        req_matrix.drop(index, inplace=True) 

req_matrix.to_csv('combined_file_size3.csv')
     

In [3]:
import pandas as pd
# prune reqs with size > 3:
req_matrix = pd.read_csv('combined_file.csv')
req_matrix.drop(req_matrix.columns[0], axis=1, inplace=True)
req_matrix.to_csv('combined_file_size3.csv')